In [20]:
import pandas as pd
import numpy as np
import json
from google.cloud import bigquery

In [21]:
#conversion du fichier json en json line delimiter pour utiliser pd.read_json avec une chunk size. (Nombre de ligne processée en une fois)
#!cat ../raw_data/export_23-5_minified.json | jq -c '.[]' > ../raw_data/export_converted_2.json

In [22]:
#Nombre de lignes totales dans le JSON : 1320229

In [23]:
#Localisation du projet name et table_id sur gcp
project="marbotic"
table_id = "marbotic_dataset.event_properties"

In [24]:
#intégration des credentials 
from google.oauth2 import service_account

key_path = "/Users/antonin/code/AntoninAnq/gcp/marbotic-7d02fac30bd8.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [25]:
# Observation de la colonne Games, on a des listes de jeu. Sur le chunk de 100K on a 158 combinaisons différentes
tmp = pd.read_csv('../raw_data/event_properties.csv')
#tmp

/var/folders/c7/kdqvz76x679f28lhd235r_v40000gn/T/ipykernel_77953/2715471165.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv('../raw_data/event_properties.csv')


In [26]:
tmp.columns

Index(['Newsletter', 'Products', 'Pieces', 'Type', 'Country', 'Games',
       'Language', 'ProfileType2', 'ReceiveNewsletter', 'LoggedIn'],
      dtype='object')

In [27]:
columns_rename = {'[Game] Code':'Game_Code','[Profile] Age (days)':'Profile_Age_days','[Scene] Name':'Scene_Name',
                 '[Game] Language':'Game_Language', '[Scene] Previous':'Scene_Previous', '[Time] Spent':'Time_Spent',
                  '[Scene] Next':'Scene_Next', '[Action] Element Type':'Action_Element Type',
                  '[Scene] Section':'Scene_Section', '[Action] Element Name':'Action_Element Name',
                 '[Activity] Name':'Activity_Name', '[Scaffolding] Level':'Scaffolding_Level',
                  '[Game] Piece Code':'Game_Piece Code','[Activity] Nb Wrong Answer':'Activity_Nb Wrong Answer',
                  '[Activity] Solved':'Activity_Solved','[Activity] Piece Stamped':'Activity_Piece Stamped',
                  '[Activity] Piece Asked':'Activity_Piece Asked','[Activity] Modality':'Activity_Modality',
                  '[Time] Slot':'Time_Slot', '[Error] Type':'Error_Type',
                  '[Activation] Game Code':'Activation_Game Code',
                  '[Activation] Product Code':'Activation_Product Code',
                 '[Activation] Piece Code':'Activation_Piece Code', '[Popup] Name':'Popup_Name'}

In [28]:
CHUNKSIZE = 100000

file_path = '../raw_data/export_converted_2.json'
df = pd.read_json(file_path, lines=True, chunksize=CHUNKSIZE) #Chunk size pour avoir 14 chunk

client = bigquery.Client(project,credentials)

for i,c in enumerate(df):
    index_init = np.linspace(1,len(c),len(c),dtype='int32')
    ep=pd.DataFrame()
    c.reset_index(inplace=True)
    ep=pd.DataFrame(c["user_properties"].to_list()).copy()
    ep.drop(['Products','Pieces','Games'],axis=1,inplace=True)
    ep['event_id']=c["event_id"].copy()
    ep['id'] = (i*CHUNKSIZE) + index_init
    ep.rename(columns=columns_rename)
    #ep.to_csv('../raw_data/event_properties.csv',index=False)
    job = client.load_table_from_dataframe(ep, table_id)  # Make an API request. ,job_config=job_config
    job.result()  # Wait for the job to complete.
    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

Loaded 100000 rows and 9 columns to marbotic_dataset.event_properties
Loaded 200000 rows and 9 columns to marbotic_dataset.event_properties
Loaded 300000 rows and 9 columns to marbotic_dataset.event_properties
Loaded 400000 rows and 9 columns to marbotic_dataset.event_properties
Loaded 500000 rows and 9 columns to marbotic_dataset.event_properties
Loaded 600000 rows and 9 columns to marbotic_dataset.event_properties
Loaded 700000 rows and 9 columns to marbotic_dataset.event_properties
Loaded 800000 rows and 9 columns to marbotic_dataset.event_properties
Loaded 900000 rows and 9 columns to marbotic_dataset.event_properties
Loaded 1000000 rows and 9 columns to marbotic_dataset.event_properties
Loaded 1100000 rows and 9 columns to marbotic_dataset.event_properties
Loaded 1200000 rows and 9 columns to marbotic_dataset.event_properties
Loaded 1300000 rows and 9 columns to marbotic_dataset.event_properties
Loaded 1320229 rows and 9 columns to marbotic_dataset.event_properties
